This IPython notebook illustrates how to perform blocking using Attribute equivalence blocker.

In [1]:
# Import py_entitymatching package
import py_entitymatching as em
import os

/Users/pradap/miniconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Read Input Tables

In [2]:
# Get the datasets directory
datasets_dir = em.get_install_path() + os.sep + 'datasets'

# Get the paths of the input tables
path_A = datasets_dir + os.sep + 'person_table_A.csv'
path_B = datasets_dir + os.sep + 'person_table_B.csv'

In [3]:
# Read the CSV files and set 'ID' as the key attribute
A = em.read_csv_metadata(path_A, key='ID')
B = em.read_csv_metadata(path_B, key='ID')

In [4]:
A.head()

,ID,name,birth_year,hourly_wage,address,zipcode
0,a1,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107
1,a2,Michael Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
2,a3,William Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
3,a4,Binto George,1987,32.5,"423 Powell St, San Francisco",94122
4,a5,Alphonse Kemper,1984,35.0,"1702 Post Street, San Francisco",94122


In [5]:
B.head()

,ID,name,birth_year,hourly_wage,address,zipcode
0,b1,Mark Levene,1987,29.5,"108 Clement St, San Francisco",94107
1,b2,Bill Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
2,b3,Mike Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
3,b4,Joseph Kuan,1982,26.0,"108 South Park, San Francisco",94122
4,b5,Alfons Kemper,1984,35.0,"170 Post St, Apt 4, San Francisco",94122


## Ways To Do Attribute Equivalence Blocking

There are three different ways to do attribute equivalence blocking:

1. Block two tables to produce a `candidate set` of tuple pairs.
2. Block a `candidate set` of tuple pairs to typically produce a reduced candidate set of tuple pairs.
3. Block two tuples to check if a tuple pair would get blocked.

### Block Tables to Produce a Candidate Set of Tuple Pairs

In [6]:
# Instantiate attribute equivalence blocker object
ab = em.AttrEquivalenceBlocker()

For the given two tables, we will assume that two persons with different `zipcode` value do not refer to the same real world person. So, we apply attribute equivalence blocking on `zipcode`. That is, we block all the tuple pairs that have different zipcodes.

In [7]:
# Use block_tables to apply blocking over two input tables.
C1 = ab.block_tables(A, B, 
                    l_block_attr='zipcode', r_block_attr='zipcode', 
                    l_output_attrs=['name', 'birth_year', 'zipcode'],
                    r_output_attrs=['name', 'birth_year', 'zipcode'],
                    l_output_prefix='l_', r_output_prefix='r_')

In [8]:
# Display the candidate set of tuple pairs
C1.head()

,_id,l_ID,r_ID,l_name,l_birth_year,l_zipcode,r_name,r_birth_year,r_zipcode
0,0,a1,b1,Kevin Smith,1989,94107,Mark Levene,1987,94107
1,1,a1,b2,Kevin Smith,1989,94107,Bill Bridge,1986,94107
2,2,a1,b6,Kevin Smith,1989,94107,Michael Brodie,1987,94107
3,3,a3,b1,William Bridge,1986,94107,Mark Levene,1987,94107
4,4,a3,b2,William Bridge,1986,94107,Bill Bridge,1986,94107


Note that the tuple pairs in the candidate set have the same zipcode. 

The attributes included in the candidate set are based on l_output_attrs and r_output_attrs mentioned in block_tables command (the key columns are included by default). Specifically, the list of attributes mentioned in l_output_attrs are picked from table A and the list of attributes mentioned in r_output_attrs are picked from table B. The attributes in the candidate set are prefixed based on l_output_prefix and r_ouptut_prefix parameter values mentioned in block_tables command.

In [9]:
# Show the metadata of C1
em.show_properties(C1)

id: 4642563744
rtable(obj.id): 4642399176
ltable(obj.id): 4642398392
key: _id
fk_ltable: l_ID
fk_rtable: r_ID


In [10]:
id(A), id(B)

(4642398392, 4642399176)

Note that the metadata of C1 includes key, foreign key to the left and right tables (i.e A and B) and pointers to left and right tables.

### Block a Candidate Set of Tuple Pairs

In the above, we see that the candidate set produced after blocking over input tables include tuple pairs that have different birth years. We will assume that two persons with different birth years cannot refer to the same person. So, we block the candidate set of tuple pairs on `birth_year`. That is, we block all the tuple pairs that have different birth years.

In [11]:
# Instantiate Attr. Equivalence Blocker
ab = em.AttrEquivalenceBlocker()
# Use block_tables to apply blocking over two input tables.
C2 = ab.block_candset(C1, l_block_attr='birth_year', r_block_attr='birth_year')

0%           100%
[###############] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [12]:
C2.head()

,_id,l_ID,r_ID,l_name,l_birth_year,l_zipcode,r_name,r_birth_year,r_zipcode
4,4,a3,b2,William Bridge,1986,94107,Bill Bridge,1986,94107
6,6,a2,b3,Michael Franklin,1988,94122,Mike Franklin,1988,94122
14,14,a5,b5,Alphonse Kemper,1984,94122,Alfons Kemper,1984,94122


Note that, the tuple pairs in the resulting candidate set have the same birth year. 

The attributes included in the resulting candidate set are based on the input candidate set (i.e the same attributes are retained).

In [13]:
# Show the metadata of C1
em.show_properties(C2)

id: 4642400128
rtable(obj.id): 4642399176
ltable(obj.id): 4642398392
key: _id
fk_ltable: l_ID
fk_rtable: r_ID


In [14]:
id(A), id(B)

(4642398392, 4642399176)

As we saw earlier the metadata of C2 includes the same metadata as C1. That is, it includes key, foreign key to the left and right tables (i.e A and B) and pointers to left and right tables.

### Block Two tuples To Check If a Tuple Pair Would Get Blocked

We can apply attribute equivalence blocking to a tuple pair to check if it is going to get blocked. For example, we can check if the first tuple from A and B will get blocked if we block on `zipcode`.

In [15]:
# Display the first tuple from table A
A.ix[[0]]

,ID,name,birth_year,hourly_wage,address,zipcode
0,a1,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107


In [16]:
# Display the first tuple from table B
B.ix[[0]]

,ID,name,birth_year,hourly_wage,address,zipcode
0,b1,Mark Levene,1987,29.5,"108 Clement St, San Francisco",94107


In [17]:
# Instantiate Attr. Equivalence Blocker
ab = em.AttrEquivalenceBlocker()

# Apply blocking to a tuple pair from the input tables on zipcode and get blocking status
status = ab.block_tuples(A.ix[0], B.ix[0], l_block_attr='zipcode', r_block_attr='zipcode')

# Print the blocking status
print(status)

False


The above result says that the tuple pair will not be blocked, i.e. this tuple pair will be included in the candidate set.